# Floor Price Calculation: Approach Analysis

This notebook investigates three different approaches to calculating floor prices:

1. **SQL Floor** - Average of 4 lowest recent SALES
2. **Lowest Ask** - Cheapest ACTIVE listing
3. **Order Book Floor** - Deepest bucket from ACTIVE listings (OrderBookAnalyzer)

## Goals
- Compare floor estimates across cards with varying data quality
- Identify when each approach is most accurate
- Recommend a unified pricing strategy

In [ ]:
import sys
sys.path.insert(0, '..')

import pandas as pd
import numpy as np
from datetime import datetime, timedelta, timezone
from sqlmodel import Session, select, func, and_
from app.db import engine
from app.models import Card, MarketPrice, BlokpaxListing
from app.services.order_book import OrderBookAnalyzer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [ ]:
# First, let's check the data situation
with Session(engine) as session:
    from datetime import datetime, timedelta, timezone
    
    today = datetime.now(timezone.utc).replace(hour=0, minute=0, second=0, microsecond=0)
    
    print("=" * 70)
    print("DATA REALITY CHECK")
    print("=" * 70)
    
    # Total counts
    total_cards = session.execute(select(func.count(Card.id))).scalar()
    total_active = session.execute(
        select(func.count(MarketPrice.id)).where(MarketPrice.listing_type == 'active')
    ).scalar()
    
    print(f"\nTotal Cards: {total_cards}")
    print(f"Active Listings: {total_active:,}")
    
    # Sales by week
    print("\nSales by Week (recent):")
    print("-" * 40)
    for weeks_ago in range(6):
        start = today - timedelta(weeks=weeks_ago+1)
        end = today - timedelta(weeks=weeks_ago)
        count = session.execute(
            select(func.count(MarketPrice.id)).where(
                MarketPrice.listing_type == 'sold',
                MarketPrice.sold_date >= start,
                MarketPrice.sold_date < end,
            )
        ).scalar()
        bar = '#' * min(count // 3, 30)
        week_label = start.strftime('%b %d') + ' - ' + end.strftime('%b %d')
        print(f"  {week_label}: {count:4} {bar}")
    
    # Cards with active listings
    cards_active = session.execute(
        select(func.count(func.distinct(MarketPrice.card_id)))
        .where(MarketPrice.listing_type == 'active')
    ).scalar()
    print(f"\nCards with active listings: {cards_active} ({cards_active/total_cards*100:.1f}%)")
    
    print("\n⚠️  NOTE: Recent sales volume is low (holiday period).")
    print("    This makes SQL Floor less reliable than Order Book Floor.")

## 1. Define Floor Calculation Functions

In [ ]:
def get_sql_floor(session: Session, card_id: int, days: int = 30, treatment: str | None = None) -> dict:
    """
    SQL Floor: Average of 4 lowest recent SALES.
    This is the current approach used in card list/detail views.
    """
    cutoff = datetime.now(timezone.utc) - timedelta(days=days)
    
    filters = [
        MarketPrice.card_id == card_id,
        MarketPrice.listing_type == 'sold',
        MarketPrice.price.isnot(None),
        MarketPrice.sold_date >= cutoff,
    ]
    if treatment:
        filters.append(MarketPrice.treatment == treatment)
    
    # Get 4 lowest sales
    stmt = (
        select(MarketPrice.price)
        .where(and_(*filters))
        .order_by(MarketPrice.price.asc())
        .limit(4)
    )
    prices = [r for r in session.execute(stmt).scalars().all()]
    
    if not prices:
        return {'floor': None, 'sample_size': 0, 'method': 'sql_floor'}
    
    return {
        'floor': np.mean(prices),
        'sample_size': len(prices),
        'prices': prices,
        'method': 'sql_floor'
    }


def get_lowest_ask(session: Session, card_id: int, treatment: str | None = None) -> dict:
    """
    Lowest Ask: Cheapest ACTIVE listing.
    Currently shown separately in card detail view.
    """
    filters = [
        MarketPrice.card_id == card_id,
        MarketPrice.listing_type == 'active',
        MarketPrice.price.isnot(None),
    ]
    if treatment:
        filters.append(MarketPrice.treatment == treatment)
    
    stmt = (
        select(func.min(MarketPrice.price), func.count(MarketPrice.id))
        .where(and_(*filters))
    )
    result = session.execute(stmt).first()
    
    return {
        'floor': result[0] if result else None,
        'listing_count': result[1] if result else 0,
        'method': 'lowest_ask'
    }


def get_order_book_floor(session: Session, card_id: int, days: int = 30, treatment: str | None = None) -> dict:
    """
    Order Book Floor: Deepest bucket from ACTIVE listings.
    Uses OrderBookAnalyzer with adaptive bucketing.
    """
    analyzer = OrderBookAnalyzer(session)
    result = analyzer.estimate_floor(card_id=card_id, treatment=treatment, days=days)
    
    if not result:
        return {'floor': None, 'confidence': 0, 'method': 'order_book'}
    
    return {
        'floor': result.floor_estimate,
        'confidence': result.confidence,
        'source': result.source,
        'total_listings': result.total_listings,
        'deepest_bucket': result.deepest_bucket,
        'method': 'order_book'
    }

## 2. Sample Cards with Varying Data Quality

In [ ]:
with Session(engine) as session:
    # Get cards with sales counts for stratified sampling
    cutoff = datetime.now(timezone.utc) - timedelta(days=30)
    
    stmt = (
        select(
            Card.id,
            Card.name,
            Card.rarity,
            func.count(MarketPrice.id).filter(
                and_(MarketPrice.listing_type == 'sold', MarketPrice.sold_date >= cutoff)
            ).label('sales_30d'),
            func.count(MarketPrice.id).filter(
                MarketPrice.listing_type == 'active'
            ).label('active_listings'),
        )
        .join(MarketPrice, Card.id == MarketPrice.card_id, isouter=True)
        .group_by(Card.id, Card.name, Card.rarity)
        .order_by(func.count(MarketPrice.id).desc())
    )
    
    results = session.execute(stmt).all()
    card_df = pd.DataFrame(results, columns=['card_id', 'name', 'rarity', 'sales_30d', 'active_listings'])

print(f"Total cards: {len(card_df)}")
print(f"\nCards by data quality:")
print(f"  High data (>=10 sales, >=5 active): {len(card_df[(card_df.sales_30d >= 10) & (card_df.active_listings >= 5)])}")
print(f"  Medium data (5-9 sales, 2-4 active): {len(card_df[(card_df.sales_30d.between(5, 9)) & (card_df.active_listings.between(2, 4))])}")
print(f"  Low data (<5 sales OR <2 active): {len(card_df[(card_df.sales_30d < 5) | (card_df.active_listings < 2)])}")

card_df.head(20)

## 3. Compare Floor Calculations Across Cards

In [ ]:
# Select sample cards from each data quality tier
high_data = card_df[(card_df.sales_30d >= 10) & (card_df.active_listings >= 5)].head(5)
medium_data = card_df[(card_df.sales_30d.between(3, 9)) & (card_df.active_listings >= 2)].head(5)
low_data = card_df[(card_df.sales_30d < 3) | (card_df.active_listings < 2)].head(5)

sample_cards = pd.concat([high_data, medium_data, low_data]).drop_duplicates()
print(f"Analyzing {len(sample_cards)} sample cards...")

comparison_data = []

with Session(engine) as session:
    for _, row in sample_cards.iterrows():
        card_id = int(row['card_id'])
        
        sql_floor = get_sql_floor(session, card_id)
        lowest_ask = get_lowest_ask(session, card_id)
        order_book = get_order_book_floor(session, card_id)
        
        comparison_data.append({
            'card_id': card_id,
            'name': row['name'],
            'rarity': row['rarity'],
            'sales_30d': row['sales_30d'],
            'active_listings': row['active_listings'],
            # SQL Floor
            'sql_floor': sql_floor['floor'],
            'sql_sample': sql_floor['sample_size'],
            # Lowest Ask
            'lowest_ask': lowest_ask['floor'],
            'ask_count': lowest_ask['listing_count'],
            # Order Book
            'ob_floor': order_book['floor'],
            'ob_confidence': order_book.get('confidence', 0),
            'ob_source': order_book.get('source', 'N/A'),
        })

comparison_df = pd.DataFrame(comparison_data)
comparison_df

## 4. Analyze Differences Between Approaches

In [ ]:
# Calculate differences between approaches
df = comparison_df.copy()

# Difference: SQL Floor vs Order Book
df['sql_vs_ob'] = df['sql_floor'] - df['ob_floor']
df['sql_vs_ob_pct'] = ((df['sql_floor'] - df['ob_floor']) / df['ob_floor'] * 100).round(1)

# Difference: Lowest Ask vs Order Book
df['ask_vs_ob'] = df['lowest_ask'] - df['ob_floor']
df['ask_vs_ob_pct'] = ((df['lowest_ask'] - df['ob_floor']) / df['ob_floor'] * 100).round(1)

# Difference: SQL Floor vs Lowest Ask
df['sql_vs_ask'] = df['sql_floor'] - df['lowest_ask']
df['sql_vs_ask_pct'] = ((df['sql_floor'] - df['lowest_ask']) / df['lowest_ask'] * 100).round(1)

print("Price Differences Analysis:")
print("="*60)
print(f"\nSQL Floor vs Order Book Floor:")
print(f"  Mean difference: ${df['sql_vs_ob'].mean():.2f} ({df['sql_vs_ob_pct'].mean():.1f}%)")
print(f"  Median difference: ${df['sql_vs_ob'].median():.2f}")

print(f"\nLowest Ask vs Order Book Floor:")
print(f"  Mean difference: ${df['ask_vs_ob'].mean():.2f} ({df['ask_vs_ob_pct'].mean():.1f}%)")
print(f"  Median difference: ${df['ask_vs_ob'].median():.2f}")

print(f"\nSQL Floor vs Lowest Ask:")
print(f"  Mean difference: ${df['sql_vs_ask'].mean():.2f} ({df['sql_vs_ask_pct'].mean():.1f}%)")
print(f"  Median difference: ${df['sql_vs_ask'].median():.2f}")

df[['name', 'sql_floor', 'lowest_ask', 'ob_floor', 'ob_confidence', 'sql_vs_ob_pct', 'ask_vs_ob_pct']]

## 5. When Does Each Approach Work Best?

In [ ]:
print("APPROACH ANALYSIS")
print("="*70)

print("\n📊 SQL FLOOR (Avg of 4 lowest sales)")
print("-"*40)
print("✅ Pros:")
print("   - Based on ACTUAL transactions (real market clearing price)")
print("   - Less susceptible to outlier listings")
print("   - Works when no active listings exist")
print("❌ Cons:")
print("   - Backward-looking (stale in fast-moving markets)")
print("   - Requires 4+ sales for accurate average")
print("   - Can be skewed by sniped/auction deals")

print("\n📋 LOWEST ASK (Cheapest active listing)")
print("-"*40)
print("✅ Pros:")
print("   - Most current price signal")
print("   - What a buyer would actually pay right now")
print("   - Simple and intuitive")
print("❌ Cons:")
print("   - Single outlier can distort (mispriced listing)")
print("   - Doesn't account for listing depth")
print("   - May be unrealistic (damaged, incomplete, etc.)")

print("\n📈 ORDER BOOK FLOOR (Deepest bucket)")
print("-"*40)
print("✅ Pros:")
print("   - Considers DEPTH (where most liquidity sits)")
print("   - Filters outliers via bucketing")
print("   - Includes confidence score")
print("   - Better represents 'true' floor with liquidity")
print("❌ Cons:")
print("   - Requires sufficient active listings")
print("   - Falls back to sales when no listings")
print("   - Bucket boundaries can be arbitrary")

## 6. Recommendation: Hybrid Approach

In [ ]:
def get_hybrid_floor(
    session: Session,
    card_id: int,
    days: int = 30,
    treatment: str | None = None,
    confidence_threshold: float = 0.5,
) -> dict:
    """
    Hybrid Floor: Use Order Book when confident, fall back to SQL Floor.
    
    Logic:
    1. If Order Book confidence >= threshold: use Order Book floor
    2. Else: use SQL floor (avg of 4 lowest sales)
    3. Always show Lowest Ask separately as "buy now" price
    """
    order_book = get_order_book_floor(session, card_id, days, treatment)
    sql_floor = get_sql_floor(session, card_id, days, treatment)
    lowest_ask = get_lowest_ask(session, card_id, treatment)
    
    # Decision logic
    if order_book['floor'] is not None and order_book.get('confidence', 0) >= confidence_threshold:
        primary_floor = order_book['floor']
        primary_source = 'order_book'
        primary_confidence = order_book['confidence']
    elif sql_floor['floor'] is not None:
        primary_floor = sql_floor['floor']
        primary_source = 'sql_floor'
        primary_confidence = min(0.4, sql_floor['sample_size'] / 4 * 0.4)  # Max 0.4 for SQL
    else:
        primary_floor = lowest_ask['floor']
        primary_source = 'lowest_ask'
        primary_confidence = 0.2  # Low confidence for single listing
    
    return {
        'floor': primary_floor,
        'source': primary_source,
        'confidence': primary_confidence,
        'lowest_ask': lowest_ask['floor'],
        'all_estimates': {
            'order_book': order_book,
            'sql_floor': sql_floor,
            'lowest_ask': lowest_ask,
        }
    }


# Test hybrid approach on sample cards
print("HYBRID FLOOR RESULTS")
print("="*80)

hybrid_data = []
with Session(engine) as session:
    for _, row in sample_cards.iterrows():
        card_id = int(row['card_id'])
        result = get_hybrid_floor(session, card_id, confidence_threshold=0.5)
        
        hybrid_data.append({
            'name': row['name'][:25],
            'hybrid_floor': result['floor'],
            'source': result['source'],
            'confidence': result['confidence'],
            'lowest_ask': result['lowest_ask'],
        })

hybrid_df = pd.DataFrame(hybrid_data)
print(hybrid_df.to_string(index=False))

print(f"\n\nSource distribution:")
print(hybrid_df['source'].value_counts())

## 7. Deep Dive: Treatment-Level Analysis

In [ ]:
# Pick a high-volume card for treatment analysis
target_card = sample_cards[sample_cards.sales_30d == sample_cards.sales_30d.max()].iloc[0]
print(f"Analyzing: {target_card['name']} (ID: {target_card['card_id']})")
print(f"Data: {target_card['sales_30d']} sales, {target_card['active_listings']} active listings")
print("="*70)

treatments = ['Classic Paper', 'Classic Foil', 'Stonefoil', 'Formless Foil', 'Prerelease', 'Promo']

treatment_data = []
with Session(engine) as session:
    for treatment in treatments:
        hybrid = get_hybrid_floor(session, int(target_card['card_id']), treatment=treatment)
        
        treatment_data.append({
            'treatment': treatment,
            'hybrid_floor': hybrid['floor'],
            'source': hybrid['source'],
            'confidence': hybrid['confidence'],
            'lowest_ask': hybrid['lowest_ask'],
            'ob_floor': hybrid['all_estimates']['order_book']['floor'],
            'sql_floor': hybrid['all_estimates']['sql_floor']['floor'],
        })

treatment_df = pd.DataFrame(treatment_data)
print("\nFloor by Treatment (Hybrid Approach):")
print(treatment_df.to_string(index=False))

## 8. Summary and Recommendations

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════╗
║                        FLOOR PRICE RECOMMENDATIONS                          ║
╠══════════════════════════════════════════════════════════════════════════════╣
║                                                                              ║
║  OPTION A: Keep Separate (Current State)                                    ║
║  ─────────────────────────────────────────                                  ║
║  • SQL Floor shown in card list/detail ("Floor: $X")                        ║
║  • Order Book Floor shown separately with confidence                         ║
║  • Lowest Ask shown as "Buy Now" price                                       ║
║  PRO: No changes needed, users see all data points                          ║
║  CON: Confusing - which floor is "real"?                                    ║
║                                                                              ║
║  OPTION B: Replace with Order Book                                           ║
║  ─────────────────────────────────────                                       ║
║  • Use Order Book floor as primary                                           ║
║  • Fall back to SQL floor when confidence < 50%                             ║
║  • Always show confidence indicator                                          ║
║  PRO: Single source of truth, forward-looking                               ║
║  CON: Requires sufficient active listings, more complex                      ║
║                                                                              ║
║  OPTION C: Hybrid (RECOMMENDED)                                              ║
║  ──────────────────────────────                                              ║
║  • PRIMARY: Order Book floor when confidence >= 50%                         ║
║  • FALLBACK: SQL floor when low confidence                                  ║
║  • SEPARATE: Lowest Ask always shown as "instant buy"                       ║
║  • TRANSPARENT: Show source and confidence                                   ║
║                                                                              ║
║  Implementation:                                                             ║
║    1. Create unified get_floor() function                                   ║
║    2. Update card API to use hybrid                                         ║
║    3. UI shows: "Floor: $X.XX (85% conf)" + "Buy Now: $Y.YY"                ║
║    4. Tooltip explains data source                                          ║
║                                                                              ║
╚══════════════════════════════════════════════════════════════════════════════╝
""")

In [ ]:
# Implementation sketch for hybrid floor
print("""
IMPLEMENTATION SKETCH: Unified Floor Service
============================================

```python
# app/services/floor_pricing.py

class FloorPricingService:
    def __init__(self, session: Session):
        self.session = session
        self.order_book = OrderBookAnalyzer(session)
    
    def get_floor(
        self,
        card_id: int,
        treatment: str | None = None,
        days: int = 30,
    ) -> FloorEstimate:
        # 1. Try Order Book first
        ob_result = self.order_book.estimate_floor(card_id, treatment, days)
        
        if ob_result and ob_result.confidence >= 0.5:
            return FloorEstimate(
                floor=ob_result.floor_estimate,
                confidence=ob_result.confidence,
                source='order_book',
                depth=ob_result.total_listings,
            )
        
        # 2. Fall back to SQL floor
        sql_floor = self._get_sql_floor(card_id, treatment, days)
        if sql_floor:
            return FloorEstimate(
                floor=sql_floor,
                confidence=0.4,  # Lower confidence for historical
                source='sales_history',
            )
        
        # 3. Last resort: lowest ask
        lowest = self._get_lowest_ask(card_id, treatment)
        return FloorEstimate(
            floor=lowest,
            confidence=0.2,
            source='single_listing',
        )
```
""")